In [1]:
#Libraries
import pandas as pd
import re
import requests

import os
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import config 

from getpass import getpass

import random

from selenium.webdriver.common.action_chains import ActionChains

In [2]:
## first we need to initiate the driver##
driver = webdriver.Chrome('C:\\Users\\catar\\OneDrive\\Documentos\\chromedriver.exe')

In [3]:
# open the website
driver.get('https://www.glassdoor.com')

In [4]:
# Login in
signin = driver.find_elements_by_css_selector("button[class^='d-flex align-items-center justify-content-center order-1 order-md-2 mr-auto mr-md-0 p-0 LockedHomeHeaderStyles__signInButton']")[0].click()

time.sleep(2.5) ## sleep timer in order to mimic human interaction

In [5]:
# Insert email
email = getpass('Enter your email') ## input email

email_box = driver.find_element_by_id("userEmail") ## box to input email

email_box.send_keys(email)  ## send email to webpage

time.sleep(3.5) ## sleep timer in order to mimic human interaction

In [6]:
#Insert password

password = getpass('Enter your password') ## input password

pass_box = driver.find_element_by_id('userPassword') ## box to input password

pass_box.send_keys(password)  ## send password to webpage

time.sleep(1.3) ## sleep timer in order to mimic human interaction

In [7]:
#login

login = driver.find_elements_by_css_selector("button[class^='gd-ui-button minWidthBtn css-8i7bc2']")[0].click()

time.sleep(1.6)  ## sleep timer in order to mimic human interaction


In [8]:
#######################################

In [31]:
# Lets try a function for that

def interview_info(job_title,location):
    
    actions = ActionChains(driver)
    
    # first lets go to interview page
    interview_icon = "//a[@data-test = 'site-header-interviews']" 
    interview_button = driver.find_element_by_xpath(interview_icon)
    interview_click = interview_button.click()
    time.sleep(random.uniform(5,7)) ## sleep timer in order to mimic human interaction
    
    # lets input the job title to search
    job_title_box = driver.find_element_by_id("KeywordSearch") ## box to input job title
    job_title_box.send_keys(job_title) ## send job title to page
    time.sleep(random.uniform(5,7)) ## sleep timer in order to mimic human interaction
    
    # lets input the location to search
    location_box = driver.find_element_by_id("LocationSearch")  ## box to input job title
    location_box.clear() ##clear initial input
    location_box.send_keys(location) ## send job title to page
    time.sleep(random.uniform(5,7)) ## sleep timer in order to mimic human interaction
    
    # find information
    interview_search = driver.find_element_by_id('HeroSearchButton') ## click search button
    interview_search.click()  ## click search button
    time.sleep(random.uniform(5,7))  ## sleep timer in order to mimic human interaction
    
    ## Now that we have the results lets check total number of results and page of results
    results = "//div[@class='reviewCount center']"
    total_results = driver.find_element_by_xpath(results).text
    number_of_results = int(total_results)
    number_of_pages = round(number_of_results/10)

    ## Create to add the info scrapped
    job_company = []
    questions = []
    answers = []

    ## Actions
    actions = ActionChains(driver)
    
    #lets iterate throw the questions
    for i in range(1,(number_of_results+1)):
        display(i)
        
        #company info
        xpath1 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//span[@class='authorInfo']"
        job_company_name = driver.find_element_by_xpath(xpath1)
        job_company_name_text = job_company_name.text
        display(job_company_name_text)
        job_company.append(job_company_name_text)
        time.sleep(random.uniform(3,6))

        #question info
        xpath2 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//p[@class='questionText h3']"
        questions_gd = driver.find_element_by_xpath(xpath2)
        questions_gd_text = questions_gd.text
        display(questions_gd_text)
        questions.append(questions_gd_text)
        time.sleep(random.uniform(3,6))

        #answer info
        try:
            #click answer button
            if ((i%10) != 1):   
                xpath3 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//a[@class='userResponseLink margTop block hiddenLink mmLink ']"
                answer_icon = driver.find_element_by_xpath(xpath3)
                answer_icon_click = answer_icon.click()
                time.sleep(random.uniform(3,6))         
        
            #get answer info            
            xpath4 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//div[@class='responseText padTopSm tbl fill']"
            answers_gd = driver.find_element_by_xpath(xpath4)
            answers_gd_text = answers_gd.text
            display(answers_gd_text)
            answers.append(answers_gd_text)
            time.sleep(random.uniform(3,6))
        
        #if no answer presented
        except:
            answers_gd_text = 'No answer'
            display(answers_gd_text)
            answers.append(answers_gd_text)
            time.sleep(random.uniform(3,6))
            
        #go to next page
        if ((i%10) == 0):
            xpath_page = "//div[@class='pagingControls cell middle']//li[@class ='next']"
            page_change = driver.find_element_by_xpath(xpath_page)
            page_change_button = page_change.click()
            time.sleep(random.uniform(15,20))
    
    
    #lets retrieve dataframe with this info
    interview_data = pd.DataFrame([job_company,questions,answers]).T
    interview_data.columns = ['job_company', 'questions','answers']
    interview_data['country'] = location
    interview_data['area'] = job_title
    display(interview_data)

    return interview_data

In [20]:
def data_to_csv(location,scrap_data):
    t = time.localtime() # check time
    m = str(t.tm_mon) #scrap month
    d = str(t.tm_mday) #scrap day
    h = str(t.tm_hour) #scrap hour
    mt = str(t.tm_min) #scrap minute

    #lets structure how to present this info in the file name
    structure = str(location_pt)+"_m"+str(m)+"_d"+str(d)+"_h"+str(h)+"_"+str(mt)

    #turn into csv
    scrap_data.to_csv("interview_data_"+structure+".csv",sep = ',',index=False)


In [22]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Senior Data Scientist at OLX Group was asked...'

'explain the reasoning in resolving the case'

'I answer all the insights I had based on data analysis and business acumen'

2

...

66

'Junior Data Scientist at FARFETCH was asked...'

"They asked me to explain the advantages of my case-study solution to a business person, who don't know anything about to Data Science and Machine Learning."

'No answer'

,job_company,questions,answers,country,area
0,Senior Data Scientist at OLX Group was asked...,explain the reasoning in resolving the case,I answer all the insights I had based on data ...,Portugal,Data Analyst
1,Data Science at MOG Technologies was asked...,"Nothing too technical, they presented one of t...",I told them that it could be solved and presen...,Portugal,Data Analyst
2,Data Scientist at Smart Consulting was asked...,Tell me about 2 projects that you seem fit to ...,Described two projetcs about AI,Portugal,Data Analyst
3,Data Scientist at Deloitte was asked...,What your motivations are?,"If the data adequately represent reality, by s...",Portugal,Data Analyst
4,Data Scientist at 360imprimir was asked...,Logic question( cases resolution),Detailed explanation about the reasoning behin...,Portugal,Data Analyst
...,...,...,...,...,...
61,Data Scientist at Builtrix was asked...,Questions related to your past work experience...,No answer,Portugal,Data Analyst
62,Data Scientist at Feedzai was asked...,"that is quite fuzzy screening test, with neces...",No answer,Portugal,Data Analyst
63,Data Scientist at Feedzai was asked...,Describe two of your past projects that had Ma...,No answer,Portugal,Data Analyst
64,Data Scientist at Miniclip was asked...,- Technical questions about the challenge,No answer,Portugal,Data Analyst


In [24]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Web Developer Junior at Prime IT was asked...'

'Seu eu sabia o que fazia a Prime IT'

'Falando o que havia lido a respeito da empresa e de seus metodos de trabalho'

2

...

31

'Full Stack Web Developer at ITSector was asked...'

'On a blank board do a simple algorithm to find a capicua.'

'No answer'

,job_company,questions,answers,country,area
0,Web Developer Junior at Prime IT was asked...,Seu eu sabia o que fazia a Prime IT,Falando o que havia lido a respeito da empresa...,Portugal,Web Developer
1,Web Developer at Leya was asked...,They asked all the usual job interview questio...,"I wrote down the general programming logic, in...",Portugal,Web Developer
2,Web Developer at Landing.jobs was asked...,If I would feel Ok with unpaid extra time ever...,Obviously not.,Portugal,Web Developer
3,Junior Web Developer at iTGROW was asked...,Difference between foreach and map Java functions,"Map returns values, foreach doesn't,...",Portugal,Web Developer
4,Web Developer at PDM&FC was asked...,I was asked about Java and JavaScript even tho...,No answer,Portugal,Web Developer
5,Web Developer at Wiremaze was asked...,What's your salary expectation?,No answer,Portugal,Web Developer
6,Web Developer at Eticadata was asked...,"Database questions, CRUD",No answer,Portugal,Web Developer
7,Web Developer at rVisual was asked...,they asked me to fix old broken code without b...,No answer,Portugal,Web Developer
8,Web Developer at Wiremaze was asked...,"I was asked to do a simple HTML page, with som...",No answer,Portugal,Web Developer
9,Web Developer at Alter Solutions was asked...,What kind of projects that you work?,No answer,Portugal,Web Developer


In [25]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX Designer at Sport Zone was asked...'

'Can you work under pressure?'

'Yes.'

2

...

11

'UI/UX Designer at OutSystems was asked...'

'Describe the UX process you are aware of.'

'No answer'

,job_company,questions,answers,country,area
0,UX Designer at Sport Zone was asked...,Can you work under pressure?,Yes.,Portugal,UX Designer
1,UX Designer at Sport Zone was asked...,Why do you want to leave you actual job?,I want a new experience.,Portugal,UX Designer
2,UX Designer at Sport Zone was asked...,Talk about you and your experience.,I just focused on my experience.,Portugal,UX Designer
3,UI & UX Designer at BNP Paribas was asked...,Previous experiences with data visualization,No answer,Portugal,UX Designer
4,UX Designer at Zalando was asked...,"The interview was more about my experience, me...",No answer,Portugal,UX Designer
5,UI/UX Designer at Pixelmatters was asked...,What is for you/your understading of UI/UX Des...,No answer,Portugal,UX Designer
6,Ui\Ux Designer at YGroup Companies was asked...,"If you had to validate this solution, how woul...",No answer,Portugal,UX Designer
7,UX Designer at Sport Zone was asked...,What do you expect from this job?,Be happy doing what I love to do. A fair salar...,Portugal,UX Designer
8,UX Designer at Feedzai was asked...,They questioned Front-End Development instead ...,No answer,Portugal,UX Designer
9,Sr UX Designer 2 at Springer Nature was asked...,"Tell me about your experience, explain your de...",No answer,Portugal,UX Designer


In [26]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UI Designer at WIT Software was asked...'

'Why do you want to join us? What motivates you?'

'To live and learn, to contribute and to evolve, because what motivates me to do this, my children.'

2

...

8

'UI/UX Designer at OutSystems was asked...'

'Describe the UX process you are aware of.'

'No answer'

,job_company,questions,answers,country,area
0,UI Designer at WIT Software was asked...,Why do you want to join us? What motivates you?,"To live and learn, to contribute and to evolve...",Portugal,UI Designer
1,UX-UI Designer at NDrive was asked...,- Why were i interested in working there and h...,No answer,Portugal,UI Designer
2,UI & UX Designer at BNP Paribas was asked...,Previous experiences with data visualization,No answer,Portugal,UI Designer
3,UI/UX Designer at Pixelmatters was asked...,What is for you/your understading of UI/UX Des...,No answer,Portugal,UI Designer
4,Ui\Ux Designer at YGroup Companies was asked...,"If you had to validate this solution, how woul...",No answer,Portugal,UI Designer
5,UX-UI Designer at BOOST IT was asked...,The interview was mostly a Q&A session based o...,No answer,Portugal,UI Designer
6,User Experience/UI Designer at Blip was asked...,"Regular questions about your method, collabora...",No answer,Portugal,UI Designer
7,UI/UX Designer at OutSystems was asked...,Describe the UX process you are aware of.,No answer,Portugal,UI Designer


In [17]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Data Analyst at Electronic Arts was asked...'

'What was one of the biggest challenge on your last work? Are you more an analytical or creative person?'

'I answered with a challenge I had when I had to start a unit from scratch.'

2

...

181

'Data Scientist at TeamCMP was asked...'

'Describe como funciona un algoritmo supervisado de ML. Describe como funciona un no supervisado. Preguntas de experiencia y como lo resolviste.'

'No answer'

,job_company,questions,answers,country
0,Data Analyst at Electronic Arts was asked...,What was one of the biggest challenge on your ...,I answered with a challenge I had when I had t...,Spain
1,Business Intelligence and Data Science Analyst...,How many universes did I managed,Really I have enough with mine ;),Spain
2,Data Scientist at Typeform was asked...,How experienced are you with SQL?,Sounds about right -- shady company.,Spain
3,Data Scientist at King was asked...,How would you implement a factorial function w...,Recursion function factorial(n) { if n = 0 ret...,Spain
4,Data Analyst/Data Scientist at Boston Consulti...,Specific bussiness related questions to a case...,Calculations. You only have 1:30 seconds per q...,Spain
...,...,...,...,...
176,Data Analyst at Morningstar was asked...,Why do you think yourself best candidate for t...,No answer,Spain
177,Data Scientist at TeamCMP was asked...,Describe como funciona un algoritmo supervisad...,No answer,Spain
178,Data Scientist at Criteo was asked...,Why Criteo? What makes Criteo different from t...,No answer,Spain
179,Analyst Data Science at Criteo was asked...,How would you explain Criteo's business model ...,No answer,Spain


In [27]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Web Developer at Glovo was asked...'

'"Imagine its your first week in the company and I ask you to do a presentation of your choice in front of your colleagues, what would it be about?"'

'My Chioce will be service render'

2

...

22

'.NET Web Developer at Electronic Arts was asked...'

'Familiar with Agile?'

'No answer'

,job_company,questions,answers,country,area
0,Web Developer at Glovo was asked...,"""Imagine its your first week in the company an...",My Chioce will be service render,Spain,Web Developer
1,Web Developer at Glovo was asked...,They asked me to explain how the Internet works,First your network must be strong and fast you...,Spain,Web Developer
2,Web Developer at Adaptive Financial Consulting...,Tell us about component lifecycle in React.,"Mentioned the most useful core methods, but th...",Spain,Web Developer
3,Junior Web Developer at Marfeel was asked...,"You don't have to say if you don't want to, bu...",I told them the range of how much it was.,Spain,Web Developer
4,Web Developer at Avature was asked...,Revisión de código + 2 ejercicios de programac...,No answer,Spain,Web Developer
5,Desarrollador Web at Denodo was asked...,Unas 30 preguntas técnicas tipo test que la ma...,No answer,Spain,Web Developer
6,Junior Web Developer at Netguru was asked...,They gave me a practical task to complete.,No answer,Spain,Web Developer
7,Desarrollador Web Junior at Sopra Steria was a...,Acerca de mis intereses e inquietudes,No answer,Spain,Web Developer
8,Full Stack Web Developer at Gamesys was asked...,Much more...,No answer,Spain,Web Developer
9,Full Stack Web Developer at Gamesys was asked...,what's immutability? explain,No answer,Spain,Web Developer


In [28]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX Designer at Hostaway was asked...'

'How will you find people for usability testing if you cannot find any existing clients locally?'

'Usability testing is better to be done with people who have not interacted with product before. So, we are escaping the biased answers and personal likes/dislikes and evaluating the usability of a system from an independent point of view.'

2

...

40

'UI/UX Designer at Kantox was asked...'

'A superficial usability analysis of their existing product.'

'No answer'

,job_company,questions,answers,country,area
0,UX Designer at Hostaway was asked...,How will you find people for usability testing...,Usability testing is better to be done with pe...,Spain,UX Designer
1,UI/UX Designer at Seocom was asked...,Since I dont know spanish he asked how much i ...,I said no but that I am a fast learner and I u...,Spain,UX Designer
2,UX Designer at GFT Technologies was asked...,They did asked difficult questions,Nothing was unexpected,Spain,UX Designer
3,UI-UX Designer at Hewlett Packard Enterprise |...,What were my current responsibilities,Saying what I do actually execute in my curren...,Spain,UX Designer
4,UX Designer at Wysiwyg was asked...,How do you propose a user test?,No answer,Spain,UX Designer
5,Diseñador Ux/Ui at KingEclient was asked...,Parece que antes de se entrevistado como tal y...,"Apenas hablé 3 minutos con la seleccionadora, ...",Spain,UX Designer
6,UX Designer at Domestika was asked...,Experiencia pasada.,No answer,Spain,UX Designer
7,UX Designer at New Work SE was asked...,How is your current team set up?,No answer,Spain,UX Designer
8,Senior UX Designer at King was asked...,Most questions were related to personal work e...,No answer,Spain,UX Designer
9,UI/UX Designer at TeamCMP was asked...,What would be your ideal working environment?,No answer,Spain,UX Designer


In [29]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX/UI Designer at Open Assessment Technologies was asked...'

'Are you able to sign an agreement with a university or a business school or any other institution?'

'No, I am not.'

2

...

41

'UI/UX Designer at Kantox was asked...'

'A superficial usability analysis of their existing product.'

'No answer'

,job_company,questions,answers,country,area
0,UX/UI Designer at Open Assessment Technologies...,Are you able to sign an agreement with a unive...,"No, I am not.",Spain,UI Designer
1,UI Designer at netzstrategen was asked...,Why do you want to change your current job.,Because I was looking for a different work cul...,Spain,UI Designer
2,Graphic/UX-UI Designer at payever was asked...,Are you okay with 10-20 usd/hour?,Are you okay working weekends?,Spain,UI Designer
3,UI/UX Designer at Seocom was asked...,Since I dont know spanish he asked how much i ...,I said no but that I am a fast learner and I u...,Spain,UI Designer
4,UI-UX Designer at Hewlett Packard Enterprise |...,What were my current responsibilities,Saying what I do actually execute in my curren...,Spain,UI Designer
5,Junior User Experience UI Designer at KingEcli...,"Personal questions, nothing unexpected.",Just being myself.,Spain,UI Designer
6,Senior UX/UI Designer at Lookiero was asked...,"Cuál ha sido mi trayectoria profesional, estud...","Con sinceridad, contando los hitos, problemas,...",Spain,UI Designer
7,UI/User Experience Designer at Ubeeqo was aske...,The questions were quite normal,No answer,Spain,UI Designer
8,Diseñador Ux/Ui at KingEclient was asked...,Parece que antes de se entrevistado como tal y...,"Apenas hablé 3 minutos con la seleccionadora, ...",Spain,UI Designer
9,UX-UI Designer at ALUXION LABS was asked...,Preguntas relacionadas al proceso de diseño + ...,No answer,Spain,UI Designer


In [30]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Germany' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Data Scientist at Kreditech Holding was asked...'

'Present a previous project and they will ask a lot of technical questions regarding the topic.'

'Hello, can you give the details of the task that was given in the test.'

2

...

150

'Data Scientist at Delivery Hero was asked...'

'After performing a linear regression, what should you look at to determine if the regression is any good?'

'No answer'

ElementClickInterceptedException: Message: element click intercepted: Element <li class="next">...</li> is not clickable at point (534, 769). Other element would receive the click: <div id="onetrust-group-container" class="ot-sdk-eight ot-sdk-columns">...</div>
  (Session info: chrome=91.0.4472.77)


In [32]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Germany' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Web Developer at NavVis was asked...'

'This greatly depends on which role you are interviewing for'

'Mechanical engineering'

2

...

66

'Part-Time Web Developer at Kolibri Games was asked...'

'What makes a good team / a good working experience?'

'No answer'

,job_company,questions,answers,country,area
0,Web Developer at NavVis was asked...,This greatly depends on which role you are int...,Mechanical engineering,Germany,Web Developer
1,Front End Web Developer at Eurowings Digital w...,Coding challenge was to integrate the Imgur AP...,"I created the application in Angular 8, along ...",Germany,Web Developer
2,Web Developer at payever was asked...,They will provide you a task and then judge yo...,"Their task setup was incomplete, very poorly c...",Germany,Web Developer
3,Web Developer at smava was asked...,Q: Experience with React? Q: Experience with s...,A: List of previous jobs with related tech.,Germany,Web Developer
4,Frontend Web Developer at Delivery Hero was as...,Js tasks,"4/5, make some errors",Germany,Web Developer
...,...,...,...,...,...
61,Senior Web Developer at Pepper was asked...,"What SOLID stands for, how would you describe ...",No answer,Germany,Web Developer
62,Full Stack Web Developer at innoscripta was as...,one of the algorithmic problems was related to...,No answer,Germany,Web Developer
63,Senior Web Developer at Zalando was asked...,Why do I want to work for them?,No answer,Germany,Web Developer
64,Web Developer at Westwing Home and Living was ...,How many squares on a chess board?,No answer,Germany,Web Developer


In [33]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Germany' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX Designer at USEEDS° was asked...'

'warum useeds'

'bisheriges lieblingsprojekt'

2

...

72

'UI-UX Designer at Wunderflats was asked...'

'What is your design process and how do you solve design problems?'

'No answer'

,job_company,questions,answers,country,area
0,UX Designer at USEEDS° was asked...,warum useeds,bisheriges lieblingsprojekt,Germany,UX Designer
1,UX Designer at Cornelsen Schulverlage was aske...,What does a typically UX proces look like for ...,I normally start out setting visions and goals...,Germany,UX Designer
2,UX Designer at Homebell was asked...,How would you change this specific design of t...,I showed a wireframe but they didnt like it to...,Germany,UX Designer
3,UX Designer at Zoi was asked...,Endlose Ausführungen über die Firmengeschichte...,Wenn ich die Website nicht selbst lesen könnte...,Germany,UX Designer
4,UI-UX Designer at REBELLE - StyleRemains was a...,What were your UI-UX responsibilities in your ...,Design and usability of the Website......,Germany,UX Designer
...,...,...,...,...,...
67,Senior UX Designer at trivago was asked...,They aked about my case study. Why I made the ...,No answer,Germany,UX Designer
68,UI/UX Designer at Global Savings Group was ask...,Why did you apply for this position?,No answer,Germany,UX Designer
69,UI-UX Designer at Gymondo was asked...,What's your role in your current team?,No answer,Germany,UX Designer
70,Senior UX Designer at Quandoo was asked...,"Generally, the questions were fairly straight-...",No answer,Germany,UX Designer


In [34]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Germany' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UI Designer at scoutbee was asked...'

'Could you describe your typical design process?'

'I gave an in-depth walk-through of my typical process, and gave examples from my specific experiences, highlighting what I can carry over to benefit the company'

2

...

84

'UI Designer at Kolibri Games was asked...'

'Superficial Questions about CV'

'No answer'

,job_company,questions,answers,country,area
0,UI Designer at scoutbee was asked...,Could you describe your typical design process?,I gave an in-depth walk-through of my typical ...,Germany,UI Designer
1,UI-UX Designer at REBELLE - StyleRemains was a...,What were your UI-UX responsibilities in your ...,Design and usability of the Website......,Germany,UI Designer
2,UI-UX Designer at Native Instruments was asked...,What do you expect from this job?,Designing interfaces and workflows.,Germany,UI Designer
3,Senior UI/User Experience Designer at Native I...,I've applied to a Senior UX/UI Designer positi...,"I've answered that, since I had applied to a U...",Germany,UI Designer
4,UI-UX Designer at Solvemate was asked...,Working for free on unpaid offsite tasks,No,Germany,UI Designer
...,...,...,...,...,...
79,UI-UX Designer at Gymondo was asked...,What's your role in your current team?,No answer,Germany,UI Designer
80,UI-UX Designer at Gymondo was asked...,What's your role in your current team?,No answer,Germany,UI Designer
81,UI-UX Designer at Wunderflats was asked...,What is your design process and how do you sol...,No answer,Germany,UI Designer
82,UI Designer at Quandoo was asked...,A few simple questions,No answer,Germany,UI Designer


In [51]:
driver.close()

#Italy
#Sweden
#United Kingdom
#Brazil
#Netherlands
#France
#Mexico

In [ ]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))